In [1]:
from delta import configure_spark_with_delta_pip, DeltaTable
from pyspark.sql import SparkSession

spark_session = (configure_spark_with_delta_pip(SparkSession.builder.master("local[*]")
                                                        .config("spark.sql.catalogImplementation", "hive")
                                                .config("spark.sql.extensions",
                                                        "io.delta.sql.DeltaSparkSessionExtension")
                                                .config("spark.sql.catalog.spark_catalog",
                                                        "org.apache.spark.sql.delta.catalog.DeltaCatalog")
                                                ).getOrCreate())

24/11/23 06:12:48 WARN Utils: Your hostname, bartosz resolves to a loopback address: 127.0.1.1; using 192.168.1.55 instead (on interface wlp0s20f3)
24/11/23 06:12:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/bartosz/.venvs/pyspark-delta-lake-jupyterlab/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/bartosz/.ivy2/cache
The jars for the packages stored in: /home/bartosz/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-36432dcc-e834-44c2-948b-62a2a7d870bd;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in local-m2-cache
:: resolution report :: resolve 373ms :: artifacts dl 18ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from local-m2-cache in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default  

**Create input tables first**

In [2]:
rm -rf ./spark-warehouse && rm -rf ./metastore_db/

In [3]:
spark_session.sql('''
          CREATE TABLE IF NOT EXISTS `default`.`versions` (
            job_version STRING NOT NULL,
            delta_table_version INT NOT NULL
          ) USING DELTA
        ''')

spark_session.sql('''
          CREATE TABLE IF NOT EXISTS `default`.`numbers_letters` (
            number INT NOT NULL,
            letter STRING NOT NULL
          ) USING DELTA
        ''')


24/11/23 06:12:56 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/11/23 06:12:56 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/11/23 06:13:00 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/11/23 06:13:00 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore bartosz@127.0.1.1
24/11/23 06:13:00 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
24/11/23 06:13:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/11/23 06:13:10 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`default`.`versions` into Hive metastor

DataFrame[]

Let's simulate the writing operations. Whenever we write, thus `MERGE` rows to the table, we update the state table.

In [4]:
from pyspark import Row

def insert_last_version_for_the_job(job_id: str):
    last_written_version_after_data_insert = (spark_session.sql('DESCRIBE HISTORY default.numbers_letters')
                            .selectExpr('MAX(version) AS last_version').collect()[0].last_version)
    new_version = (spark_session
                   .createDataFrame([Row(job_version=job_id, delta_table_version=last_written_version_after_data_insert)]))
    (DeltaTable.forName(spark_session, 'versions').alias('old_versions')
     .merge(new_version.alias('new_version'), 'old_versions.job_version = new_version.job_version')
     .whenMatchedUpdateAll().whenNotMatchedInsertAll().execute())
def merge_tables(new_numbers_letters):
    # hack as the DeltaTable doesn't support userMetadata settings at the operation level
    spark_session.conf.set("spark.databricks.delta.commitInfo.userMetadata", ";origin=merge;")
    (DeltaTable.forName(spark_session, 'numbers_letters').alias('base_table')
     .merge(new_numbers_letters.alias('new_table'), 'base_table.number = new_table.number')
     .whenMatchedUpdateAll().whenNotMatchedInsertAll().execute())

job_id = '2024-10-01'
merge_tables(spark_session.createDataFrame([Row(number=1, letter='a'), Row(number=2, letter='b'), Row(number=3, letter='c')]))
insert_last_version_for_the_job(job_id)

job_id = '2024-10-02'
merge_tables(spark_session.createDataFrame([Row(number=1, letter='A'), Row(number=4, letter='d')]))
insert_last_version_for_the_job(job_id)

job_id = '2024-10-03'
merge_tables(spark_session.createDataFrame([Row(number=1, letter='AA'), Row(number=5, letter='e'), Row(number=6, letter='f')]))
insert_last_version_for_the_job(job_id)

job_id = '2024-10-04'
merge_tables(spark_session.createDataFrame([Row(number=1, letter='AAA'), Row(number=7, letter='g'), Row(number=8, letter='h')]))
insert_last_version_for_the_job(job_id)

24/11/23 06:13:20 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


Before we merge new datasets, turns out we have to run an `OPTIMIZE` command that is outside the data orchestration layer, so it won't create a new version to the state table

In [5]:
spark_session.conf.unset('spark.databricks.delta.commitInfo.userMetadata')
DeltaTable.forName(spark_session, 'numbers_letters').optimize().executeZOrderBy('number')

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

Let's see if the table's history has now an operation different than the `MERGE`...

In [6]:
spark_session.sql('DESCRIBE HISTORY default.numbers_letters').select('version', 'timestamp', 'operation', 'userMetadata').show(truncate=False)

+-------+-----------------------+------------+--------------+
|version|timestamp              |operation   |userMetadata  |
+-------+-----------------------+------------+--------------+
|5      |2024-11-23 06:13:44.425|OPTIMIZE    |NULL          |
|4      |2024-11-23 06:13:38.787|MERGE       |;origin=merge;|
|3      |2024-11-23 06:13:32.553|MERGE       |;origin=merge;|
|2      |2024-11-23 06:13:26.562|MERGE       |;origin=merge;|
|1      |2024-11-23 06:13:17.858|MERGE       |;origin=merge;|
|0      |2024-11-23 06:13:11.171|CREATE TABLE|NULL          |
+-------+-----------------------+------------+--------------+



Let's run a `SELECT` on the numbers_letters table to see if there is no entry corresponding to the `OPTIMIZE`...

In [7]:
spark_session.sql('SELECT * FROM default.versions ORDER BY job_version DESC').show(truncate=False)

+-----------+-------------------+
|job_version|delta_table_version|
+-----------+-------------------+
|2024-10-04 |4                  |
|2024-10-03 |3                  |
|2024-10-02 |2                  |
|2024-10-01 |1                  |
+-----------+-------------------+



OK, let's run now two other `MERGE` commands...

In [8]:
job_id = '2024-10-05'
merge_tables(spark_session.createDataFrame([Row(number=1, letter='AAAA'), Row(number=9, letter='i')]))
insert_last_version_for_the_job(job_id)

job_id = '2024-10-06'
merge_tables(spark_session.createDataFrame([Row(number=10, letter='j')]))
insert_last_version_for_the_job(job_id)

**It's time to restore the table**

In [9]:
spark_session.sql('DESCRIBE HISTORY default.numbers_letters').select('version', 'timestamp', 'operation', 'userMetadata').show(truncate=False)
spark_session.sql('SELECT job_version, delta_table_version FROM versions ORDER BY job_version ASC').show(truncate=False)

scenarios = [
    ('2024-10-01', None, '# FIRST MERGE', 'Expected: TRUNCATE TABLE'),
    ('2024-10-05', '2024-10-04', 'BACKFILL# MERGE => OPTIMIZE => MERGE', 'Expected: restore to the OPTIMIZE version created after 2024-10-04'),
    ('2024-10-03', '2024-10-02', 'BACKFILL# MERGE => MERGE', 'Expected: restore to version from 2024-10-02'),
    ('2024-10-07', '2024-10-06', '# NEW INSERT', 'Expected: normal run'),
]

for currently_processed_version, previous_execution_date, label, expected_behavior in scenarios:
    print(f'=============== {label} / {currently_processed_version} ===============')
    versions = {}
    job_with_tables = spark_session.sql(f'''SELECT job_version, delta_table_version FROM versions WHERE 
                        job_version IN ("{currently_processed_version}", "{previous_execution_date}") ORDER BY job_version ASC''').collect()
    for row in job_with_tables:
        versions[row.job_version] = row.delta_table_version
    
    print(f'Versions for each run are: {versions}')
    
    versions_history = spark_session.sql('DESCRIBE HISTORY default.numbers_letters')
    versions_history.cache()
    
    last_merge_version = (versions_history.filter('operation = "MERGE"')
     .selectExpr('MAX(version) AS last_version').collect()[0].last_version)
    print(f'Last merge version was {last_merge_version}')

    maybe_previous_job_version = spark_session.sql(f'SELECT delta_table_version FROM versions WHERE job_version = "{previous_execution_date}"').collect()
    previous_job_version = None
    version_to_restore = None
    if maybe_previous_job_version:
      previous_job_version = maybe_previous_job_version[0].delta_table_version
      if previous_job_version == last_merge_version:
       print(f"Nothing to restore: previous job's version {previous_job_version} vs. last merge version {last_merge_version}")
      else:
       current_run_version = spark_session.sql(f'SELECT delta_table_version FROM versions WHERE job_version = "{currently_processed_version}"').collect()[0].delta_table_version
       version_to_restore = current_run_version - 1 
       #(DeltaTable.forName(spark_session, 'devices').restoreToVersion(previous_job_version ))
    else:
       print('Missing previous version, truncating the table')

    versions_history.unpersist()
    
    print(f'Version to restore is {version_to_restore}')
    print(expected_behavior)

+-------+-----------------------+------------+--------------+
|version|timestamp              |operation   |userMetadata  |
+-------+-----------------------+------------+--------------+
|7      |2024-11-23 06:13:52.49 |MERGE       |;origin=merge;|
|6      |2024-11-23 06:13:48.142|MERGE       |;origin=merge;|
|5      |2024-11-23 06:13:44.425|OPTIMIZE    |NULL          |
|4      |2024-11-23 06:13:38.787|MERGE       |;origin=merge;|
|3      |2024-11-23 06:13:32.553|MERGE       |;origin=merge;|
|2      |2024-11-23 06:13:26.562|MERGE       |;origin=merge;|
|1      |2024-11-23 06:13:17.858|MERGE       |;origin=merge;|
|0      |2024-11-23 06:13:11.171|CREATE TABLE|NULL          |
+-------+-----------------------+------------+--------------+

+-----------+-------------------+
|job_version|delta_table_version|
+-----------+-------------------+
|2024-10-01 |1                  |
|2024-10-02 |2                  |
|2024-10-03 |3                  |
|2024-10-04 |4                  |
|2024-10-05 |6   

## VACUUM

Let's execute a `VACUUM` operation to remove files kept by the `OPTIMIZE` operation.

In [10]:
 ls ./spark-warehouse/numbers_letters

_delta_log/
part-00000-15b8583b-52e9-4f99-9ae7-4858ecb79bcb-c000.snappy.parquet
part-00000-332c034f-8129-4738-99e7-20fc7d7aa4fd-c000.snappy.parquet
part-00000-816ce2fe-2510-4c96-9722-39791d926105-c000.snappy.parquet
part-00000-85d26a94-1d53-46b9-80e3-f137cd3d95cb-c000.snappy.parquet
part-00000-89f1cef8-8222-48b2-9f38-e1a2e3416f4d-c000.snappy.parquet
part-00000-d8644b4d-bafa-4002-82e0-dd7189f91102-c000.snappy.parquet
part-00000-f532b1f5-f052-4f4b-bb55-e2574d6bf316-c000.snappy.parquet
part-00002-27c58b71-6b5e-45a0-a2d0-32eca4141cea-c000.snappy.parquet
part-00005-22054b02-33a3-498a-978b-8cf110103684-c000.snappy.parquet
part-00007-1fd4a6e0-93f7-4d45-99b4-dd6f16149508-c000.snappy.parquet
part-00007-633dcb64-f53b-4590-8d3f-ede3457b71ec-c000.snappy.parquet


In [11]:
from delta.tables import *

spark_session.conf.set('spark.databricks.delta.retentionDurationCheck.enabled', False)
numbers_letters_delta_table = DeltaTable.forName(spark_session, 'numbers_letters')
numbers_letters_delta_table.vacuum(0)

Deleted 9 files and directories in a total of 1 directories.


DataFrame[]

In [12]:
 ls ./spark-warehouse/numbers_letters

_delta_log/
part-00000-89f1cef8-8222-48b2-9f38-e1a2e3416f4d-c000.snappy.parquet
part-00007-1fd4a6e0-93f7-4d45-99b4-dd6f16149508-c000.snappy.parquet


As you can see, 9 files are gone. Let's see if we can time-travel to each of the versions from the state table. 
In this example I'm using the _time travel_ feature but in the MERGER logic presented before, I refered to the `restore` command. The reason I'm time travelling here is that the table doesn't get materialized, i.e. there is no restore commit added, thus no new versions. Running things in isolation is an important property for any testing approach.

In [13]:
versions_history = spark_session.sql('SELECT job_version, delta_table_version FROM versions ORDER BY job_version ASC').collect()

for version_history in versions_history:
    print(f'Testing time travel for {version_history.job_version} and Delta Lake table {version_history.delta_table_version}')
    try:
        (spark_session.read.format("delta").option("versionAsOf", version_history.delta_table_version)
        .load("./spark-warehouse/numbers_letters").show(truncate=False))
        print(f'✅ Correctly time-travelled for {version_history.delta_table_version}')
    except:
        print(f"❌ Couldn't time-travel for {version_history.delta_table_version}")

Testing time travel for 2024-10-01 and Delta Lake table 1


24/11/23 06:14:34 ERROR Executor: Exception in task 0.0 in stage 436.0 (TID 13259)
org.apache.spark.SparkFileNotFoundException: File file:/home/bartosz/workspace/delta-lake-playground/017_merger/spark-warehouse/numbers_letters/part-00005-22054b02-33a3-498a-978b-8cf110103684-c000.snappy.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	

❌ Couldn't time-travel for 1
Testing time travel for 2024-10-02 and Delta Lake table 2


24/11/23 06:14:36 ERROR Executor: Exception in task 0.0 in stage 446.0 (TID 13367)
org.apache.spark.SparkFileNotFoundException: File file:/home/bartosz/workspace/delta-lake-playground/017_merger/spark-warehouse/numbers_letters/part-00000-15b8583b-52e9-4f99-9ae7-4858ecb79bcb-c000.snappy.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	

❌ Couldn't time-travel for 2
Testing time travel for 2024-10-03 and Delta Lake table 3


24/11/23 06:14:38 ERROR Executor: Exception in task 0.0 in stage 456.0 (TID 13477)
org.apache.spark.SparkFileNotFoundException: File file:/home/bartosz/workspace/delta-lake-playground/017_merger/spark-warehouse/numbers_letters/part-00000-d8644b4d-bafa-4002-82e0-dd7189f91102-c000.snappy.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	

❌ Couldn't time-travel for 3
Testing time travel for 2024-10-04 and Delta Lake table 4


24/11/23 06:14:39 ERROR Executor: Exception in task 0.0 in stage 466.0 (TID 13589)
org.apache.spark.SparkFileNotFoundException: File file:/home/bartosz/workspace/delta-lake-playground/017_merger/spark-warehouse/numbers_letters/part-00000-f532b1f5-f052-4f4b-bb55-e2574d6bf316-c000.snappy.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	

❌ Couldn't time-travel for 4
Testing time travel for 2024-10-05 and Delta Lake table 6


+------+------+
|number|letter|
+------+------+
|1     |AAAA  |
|2     |b     |
|3     |c     |
|4     |d     |
|5     |e     |
|6     |f     |
|7     |g     |
|8     |h     |
|9     |i     |
+------+------+

✅ Correctly time-travelled for 6
Testing time travel for 2024-10-06 and Delta Lake table 7
+------+------+
|number|letter|
+------+------+
|1     |AAAA  |
|2     |b     |
|3     |c     |
|4     |d     |
|5     |e     |
|6     |f     |
|7     |g     |
|8     |h     |
|9     |i     |
|10    |j     |
+------+------+

✅ Correctly time-travelled for 7


As you can notice, two last versions which are the versions after the `OPTIMIZE` operation are only available for time travel / restore.